# Desenvolvimento do modelo de score de crédito

Exploração inicial de modelos de base

In [35]:
import pandas as pd
from dagshub.data_engine import datasources
import mlflow
import dagshub
from sklearn.model_selection import train_test_split
import mlflow.sklearn
import mlflow.catboost
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import mlflow.models.signature
from mlflow.models import infer_signature
from catboost import CatBoostRegressor

## Carregando Dataset

In [ ]:
ds = datasources.get_datasource("pedromonnt/fiap-credit-score-classification-model", "processed")

INFO:gql.transport.requests:>>> {"query": "query datasource($id: ID, $name: String) {\n  datasource(id: $id, name: $name) {\n    id\n    name\n    rootUrl\n    integrationStatus\n    preprocessingStatus\n    metadataFields {\n      name\n      valueType\n      multiple\n      tags\n    }\n    type\n  }\n}", "variables": {"id": null, "name": "processed"}}
INFO:gql.transport.requests:<<< {"data":{"datasource":[{"id":5984,"name":"processed","rootUrl":"repo://pedromonnt/credit-score-classfication-model/main:data/processed","integrationStatus":"VALID","preprocessingStatus":"READY","metadataFields":[{"name":"size","valueType":"INTEGER","multiple":false,"tags":null},{"name":"media type","valueType":"STRING","multiple":false,"tags":null}],"type":"REPOSITORY"}]}}


In [24]:
ds.all().dataframe

INFO:gql.transport.requests:>>> {"query": "query datasource($id: ID, $name: String) {\n  datasource(id: $id, name: $name) {\n    id\n    name\n    rootUrl\n    integrationStatus\n    preprocessingStatus\n    metadataFields {\n      name\n      valueType\n      multiple\n      tags\n    }\n    type\n  }\n}", "variables": {"id": 5984, "name": "processed"}}
INFO:gql.transport.requests:<<< {"data":{"datasource":[{"id":5984,"name":"processed","rootUrl":"repo://pedromonnt/credit-score-classfication-model/main:data/processed","integrationStatus":"VALID","preprocessingStatus":"READY","metadataFields":[{"name":"size","valueType":"INTEGER","multiple":false,"tags":null},{"name":"media type","valueType":"STRING","multiple":false,"tags":null}],"type":"REPOSITORY"}]}}


Output()

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/pedromonnt/credit-score-classification-model "HTTP/1.1 200 OK"
INFO:gql.transport.requests:>>> {"query": "query datasourceQuery($datasource: ID!, $queryInput: QueryInput, $first: Int, $after: String) {\n  datasourceQuery(\n    datasource: $datasource\n    filter: $queryInput\n    first: $first\n    after: $after\n  ) {\n    edges {\n      node {\n        id\n        path\n        metadata {\n          key\n          value\n          timeZone\n        }\n      }\n    }\n    pageInfo {\n      hasNextPage\n      endCursor\n    }\n    queryDataTime\n    selectFields {\n      name\n      originalName\n      autoGenerated\n      valueType\n      asOf\n      multiple\n      tags\n    }\n  }\n}", "variables": {"datasource": 5984, "queryInput": {"query": null}, "first": 5000, "after": null}}


INFO:gql.transport.requests:<<< {"data":{"datasourceQuery":{"edges":[{"node":{"id":103365822,"path":"credit-score-classification-processed.csv","metadata":[{"key":"media type","value":"text/plain","timeZone":""},{"key":"size","value":24770477,"timeZone":""}]}}],"pageInfo":{"hasNextPage":false,"endCursor":null},"queryDataTime":1752629909,"selectFields":[{"name":"media type","originalName":"media type","autoGenerated":true,"valueType":"STRING","asOf":null,"multiple":false,"tags":null},{"name":"size","originalName":"size","autoGenerated":true,"valueType":"INTEGER","asOf":null,"multiple":false,"tags":null}]}}}


,path,datapoint_id,dagshub_download_url,media type,size
0,credit-score-classification-processed.csv,103365822,https://dagshub.com/api/v1/repos/pedromonnt/cr...,text/plain,24770477


INFO:httpx:HTTP Request: POST https://dagshub.com/api/internal/trackAnalyticsEvent "HTTP/1.1 200 OK"


In [ ]:
res = ds.head()

for dp in res:
    dataset_url = dp.download_url

INFO:gql.transport.requests:>>> {"query": "query datasource($id: ID, $name: String) {\n  datasource(id: $id, name: $name) {\n    id\n    name\n    rootUrl\n    integrationStatus\n    preprocessingStatus\n    metadataFields {\n      name\n      valueType\n      multiple\n      tags\n    }\n    type\n  }\n}", "variables": {"id": 5984, "name": "processed"}}
INFO:gql.transport.requests:<<< {"data":{"datasource":[{"id":5984,"name":"processed","rootUrl":"repo://pedromonnt/credit-score-classfication-model/main:data/processed","integrationStatus":"VALID","preprocessingStatus":"READY","metadataFields":[{"name":"size","valueType":"INTEGER","multiple":false,"tags":null},{"name":"media type","valueType":"STRING","multiple":false,"tags":null}],"type":"REPOSITORY"}]}}


Output()

INFO:gql.transport.requests:>>> {"query": "query datasourceQuery($datasource: ID!, $queryInput: QueryInput, $first: Int, $after: String) {\n  datasourceQuery(\n    datasource: $datasource\n    filter: $queryInput\n    first: $first\n    after: $after\n  ) {\n    edges {\n      node {\n        id\n        path\n        metadata {\n          key\n          value\n          timeZone\n        }\n      }\n    }\n    pageInfo {\n      hasNextPage\n      endCursor\n    }\n    queryDataTime\n    selectFields {\n      name\n      originalName\n      autoGenerated\n      valueType\n      asOf\n      multiple\n      tags\n    }\n  }\n}", "variables": {"datasource": 5984, "queryInput": {"query": null}, "first": 100, "after": null}}


INFO:gql.transport.requests:<<< {"data":{"datasourceQuery":{"edges":[{"node":{"id":103365822,"path":"credit-score-classification-processed.csv","metadata":[{"key":"media type","value":"text/plain","timeZone":""},{"key":"size","value":24770477,"timeZone":""}]}}],"pageInfo":{"hasNextPage":false,"endCursor":null},"queryDataTime":1752629915,"selectFields":[{"name":"media type","originalName":"media type","autoGenerated":true,"valueType":"STRING","asOf":null,"multiple":false,"tags":null},{"name":"size","originalName":"size","autoGenerated":true,"valueType":"INTEGER","asOf":null,"multiple":false,"tags":null}]}}}


INFO:httpx:HTTP Request: POST https://dagshub.com/api/internal/trackAnalyticsEvent "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://dagshub.com/api/internal/trackAnalyticsEvent "HTTP/1.1 200 OK"


In [26]:
dataset_url

'https://dagshub.com/api/v1/repos/pedromonnt/credit-score-classification-model/raw/main/data/processed/credit-score-classification-processed.csv'

In [27]:
df = pd.read_csv(dataset_url)
df.head()

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,1.0,23.0,13,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,...,1,809.98,26.822620,265.0,1,49.574949,80.415295,3,312.494089,2
1,2.0,23.0,13,19114.12,3091.000000,3.0,4.0,3.0,4.0,-1.0,...,2,809.98,31.944960,222.0,1,49.574949,118.280222,4,284.629162,2
2,3.0,-0.5,13,19114.12,3091.000000,3.0,4.0,3.0,4.0,3.0,...,2,809.98,28.609352,267.0,1,49.574949,81.699521,5,331.209863,2
3,4.0,23.0,13,19114.12,3091.000000,3.0,4.0,3.0,4.0,5.0,...,2,809.98,31.377862,268.0,1,49.574949,199.458074,6,223.451310,2
4,5.0,23.0,13,19114.12,1824.843333,3.0,4.0,3.0,4.0,6.0,...,2,809.98,24.797347,269.0,1,49.574949,41.420153,2,341.489231,2


## Desenvolvimento e experimentos de modelos

In [ ]:
dagshub.init(repo_owner="pedromonnt", repo_name="fiap-credit-score-classification-model", mlflow=True)

INFO:httpx:HTTP Request: GET https://dagshub.com/api/v1/repos/pedromonnt/credit-score-classification-model "HTTP/1.1 200 OK"


Initialized MLflow to track repo "pedromonnt/credit-score-classification-model"

INFO:dagshub:Initialized MLflow to track repo "pedromonnt/credit-score-classification-model"


Repository pedromonnt/credit-score-classification-model initialized!

INFO:dagshub:Repository pedromonnt/credit-score-classification-model initialized!


In [29]:
mlflow.autolog()

2025/07/15 22:39:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/07/15 22:39:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/15 22:39:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [30]:
# Dividir o dataset unificado e tratado em treino e teste, considerando o valor da coluna 'Credit_Score'
train_full = df[df['Credit_Score'].isin([0, 1, 2])]
test = df[~df['Credit_Score'].isin([0, 1, 2])].drop(columns='Credit_Score')

# Features e Target do conjunto de treino completo
y = train_full['Credit_Score']
X = train_full.drop(columns='Credit_Score')

# Dividir o conjunto de TREINO em treino e VALIDAÇÃO
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [32]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(test)

In [31]:
def evaluate_and_log_model(kind, model_name, model, X_test, y_test):
    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    mape = mean_absolute_percentage_error(y_test, predictions)

    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("MAPE", mape)

    signature = infer_signature(X_test, predictions)

    if kind == "catboost":
        mlflow.catboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "xgboost":
        mlflow.xgboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "lightgbm":
        mlflow.lightgbm.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    else:
        mlflow.sklearn.log_model(model, model_name, signature=signature, input_example=X_test[:5])

    print(f"Model {model_name} logged with MSE: {mse}, MAE: {mae}, R2: {r2}, MAPE: {mape}")

### Experimento com Ridge Regression

In [33]:
with mlflow.start_run(run_name="Ridge Regression"):
    param_grid = {
        'alpha': [0.1, 1.0, 10.0, 100.0],
        'fit_intercept': [True, False],
        }
    
    ridge = Ridge()

    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False))
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    mlflow.log_param("best_alpha", grid_search.best_params_['alpha'])
    mlflow.log_param("best_fit_intercept", grid_search.best_params_['fit_intercept'])

    evaluate_and_log_model("sklearn", "ridge_regression", best_model, X_val, y_val)

2025/07/15 22:39:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\pedro\OneDrive\Área de Trabalho\mba\MLOPS\credit-score-classification\credit-score-classification-model\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/15 22:39:39 WARNING mlflow.utils.autologging_utils: MLfl

Model ridge_regression logged with MSE: 0.33801345093582336, MAE: 0.4648626561199653, R2: 0.2583915952743162, MAPE: 801762218250668.9
🏃 View run Ridge Regression at: https://dagshub.com/pedromonnt/credit-score-classification-model.mlflow/#/experiments/0/runs/4e831ce652c543b19f6e58b3ad613a49
🧪 View experiment at: https://dagshub.com/pedromonnt/credit-score-classification-model.mlflow/#/experiments/0


### Decision Tree Classifier

In [36]:
with mlflow.start_run(run_name="Decision Tree Classifier"):
    param_grid = {
        'max_depth': [None, 3, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    ridge = DecisionTreeClassifier()

    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False))
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])

    evaluate_and_log_model("sklearn", "decision_tree_classifier", best_model, X_val, y_val)

2025/07/16 00:25:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\pedro\OneDrive\Área de Trabalho\mba\MLOPS\credit-score-classification\credit-score-classification-model\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/16 00:28:27 WARNING mlflow.utils.autologging_utils: MLfl

Model decision_tree_regressor logged with MSE: 0.3979, MAE: 0.3447, R2: 0.12699928531431193, MAPE: 433471464134410.3
🏃 View run Decision Tree Classifier at: https://dagshub.com/pedromonnt/credit-score-classification-model.mlflow/#/experiments/0/runs/6af5930b1bf24c6aa47e4cd831c0f5e1
🧪 View experiment at: https://dagshub.com/pedromonnt/credit-score-classification-model.mlflow/#/experiments/0


## XGBoost

In [ ]:
with mlflow.start_run(run_name="XGBoost_Classifier_Advanced"):
   
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'gamma': [0, 1],
        'reg_alpha': [0, 0.1],
        'reg_lambda': [1, 5],
        'min_child_weight': [1, 3]
    }
   
    xgb = XGBClassifier(random_state=42, verbosity=0)
    grid_search = GridSearchCV(xgb, param_grid, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False), cv=5)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_n_estimators", best_model.n_estimators)
    mlflow.log_param("best_max_depth", best_model.max_depth)
    mlflow.log_param("best_learning_rate", best_model.learning_rate)
    evaluate_and_log_model("xgboost", "XGBoost Classifier", best_model, X_test, y_test)

2025/06/24 21:50:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/michelpf/Documents/Projects/fiap-ds-mlops-10dtsr-laptop-pricing-brl/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/06/24 22:01:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered 

🏃 View run incongruous-cub-245 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/59874b38ec9744ca859723a03efd5397
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run rebellious-smelt-121 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/46a6050bb42945c5a7a99f5f3641ed6e
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run upset-pig-375 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/588b9a0b139248c4ba15118bee027df3
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run sassy-squid-688 at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/466c9f4614b246ee912b2fc548cda5c4
🧪 View 

/Users/michelpf/Documents/Projects/fiap-ds-mlops-10dtsr-laptop-pricing-brl/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/Users/michelpf/Documents/Projects/fiap-ds-mlops-10dtsr-laptop-pricing-brl/.venv/lib/python3.10/site-packages/xgboost/sklearn.py:1028: UserWarning: [22:02:15] WARNING: /Users/run

Model XGBoost Regressor logged with MSE: 2622710.25, MAE: 878.944580078125, R2: 0.6896060705184937, MAPE: 0.13514773547649384
🏃 View run XGBoost_Regressor_Advanced at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/9fcf1bcf27d946c98f857a2a9af74e26
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0


## Nova abordagem XGBoost

In [ ]:
with mlflow.start_run(run_name="XGBoost_Classifier_New"):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, 9],
        'learning_rate': [0.01, 0.1, 0.2, 0.3]
    }
    xgb = XGBClassifier(random_state=42, verbosity=0)
    grid_search = GridSearchCV(xgb, param_grid, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False), cv=5)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    mlflow.log_param("best_n_estimators", best_model.n_estimators)
    mlflow.log_param("best_max_depth", best_model.max_depth)
    mlflow.log_param("best_learning_rate", best_model.learning_rate)
    evaluate_and_log_model("xgboost_new", "XGBoost Classifier New", best_model, X_val, y_val)

2025/06/27 22:43:48 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/michelpf/Documents/Projects/fiap-ds-mlops-10dtsr-laptop-pricing-brl/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/06/27 22:44:59 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered 

Model XGBoost Regressor logged with MSE: 2609672.0, MAE: 889.0443115234375, R2: 0.6911491751670837, MAPE: 0.14123447239398956
🏃 View run XGBoost_Regressor at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/2e5d588a429649d1923868dddf7c586d
🧪 View experiment at: https://dagshub.com/michelpf/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0


## Registro de Modelo em Produção

In [ ]:
run_id = "9fcf1bcf27d946c98f857a2a9af74e26"

mlflow.register_model(model_uri=f"runs:/{run_id}/model", name="laptop-pricing-model-brl")

Registered model 'laptop-pricing-model-brl' already exists. Creating a new version of this model...
2025/06/24 22:12:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: laptop-pricing-model-brl, version 3
Created version '3' of model 'laptop-pricing-model-brl'.


<ModelVersion: aliases=[], creation_timestamp=1750813940565, current_stage='None', description='', last_updated_timestamp=1750813940565, name='laptop-pricing-model-brl', run_id='9fcf1bcf27d946c98f857a2a9af74e26', run_link='', source='mlflow-artifacts:/d4bd9539a7fb4a12aa16e3387cfaf1ac/9fcf1bcf27d946c98f857a2a9af74e26/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='3'>